<a href="https://colab.research.google.com/github/fzlfade/Praktikum-Big-Data/blob/main/Responsi/Responsi_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=8cf33f75c3714473fc9dc56c92d943787a29fe2f08020ff148a034c49a9861da
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [45]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


spark = SparkSession.builder.appName('Responsi').getOrCreate()

data = [('P001', 'Product A', 'Electronics',10,200,'2022-01-10'),
        ('P002', 'Product B', 'Electronics',30,150,'2022-03-11'),
        ('P001', 'Product C', 'Clothing',20,100,'2022-02-10'),
        ('P001', 'Product D', 'Clothing',15,120,'2022-02-12'),
        ('P001', 'Product E', 'Kitchen',25,90,'2022-04-13'),
        ('P001', 'Product F', 'Kitchen',5,300,'2022-01-14')]
columns = ['ProductID', 'ProductName', 'Category','Quantity','Price','Date']

df = spark.createDataFrame(data, schema=columns)
df.show()

#Nomor 1
df = df.withColumn('Revenue', df['Quantity'] * df['Price'])

df = df.withColumn('Month', F.date_format(F.col('Date'), 'yyyy-MM'))

result = df.groupBy('Category', 'Month') \
           .agg(F.sum('Revenue').alias('Total')) \
           .orderBy('Month', 'Category')

result.show()

#Nomor 2
maxPerCategory = df.groupBy('Category') \
                  .agg(F.max('Price').alias('PriceMax'))  \
                  .orderBy('PriceMax')
maxPerCategory.show()

#Nomor 3
# Menghitung pendapatan total untuk setiap kategori per bulan dan mengurutkan hasilnya
result = df.groupBy('Month', 'Category') \
           .agg(F.sum('Revenue').alias('TotalRevenue')) \
           .orderBy('Month', 'Category')
result.show()











import matplotlib.pyplot as plt

# Convert the Spark DataFrame to a Pandas DataFrame
pandas_df = result.toPandas()

# Plot the total revenue for each category
plt.figure(figsize=(10, 6))

# Plot data for each category
for category in pandas_df['Category'].unique():
    category_data = pandas_df[pandas_df['Category'] == category]
    plt.plot(category_data['Month'], category_data['TotalRevenue'], marker='o', label=category)

# Add titles and labels
plt.title('Total Revenue per Month for Each Category')
plt.xlabel('Date')
plt.ylabel('Total Revenue')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()

# Show the plot
plt.show()


+---------+-----------+-----------+--------+-----+----------+
|ProductID|ProductName|   Category|Quantity|Price|      Date|
+---------+-----------+-----------+--------+-----+----------+
|     P001|  Product A|Electronics|      10|  200|2022-01-10|
|     P002|  Product B|Electronics|      30|  150|2022-03-11|
|     P001|  Product C|   Clothing|      20|  100|2022-02-10|
|     P001|  Product D|   Clothing|      15|  120|2022-02-12|
|     P001|  Product E|    Kitchen|      25|   90|2022-04-13|
|     P001|  Product F|    Kitchen|       5|  300|2022-01-14|
+---------+-----------+-----------+--------+-----+----------+

+-----------+-------+-----+
|   Category|  Month|Total|
+-----------+-------+-----+
|Electronics|2022-01| 2000|
|    Kitchen|2022-01| 1500|
|   Clothing|2022-02| 3800|
|Electronics|2022-03| 4500|
|    Kitchen|2022-04| 2250|
+-----------+-------+-----+

+-----------+--------+
|   Category|PriceMax|
+-----------+--------+
|   Clothing|     120|
|Electronics|     200|
|    Kitche

KeyError: 'Date'

<Figure size 1000x600 with 0 Axes>